In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pickle
from collections import Counter
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.linear_model import (LogisticRegression,SGDClassifier)
from sklearn.metrics import log_loss,precision_score,f1_score,recall_score
from lightgbm import LGBMClassifier,LGBMRegressor
import lightgbm as lgb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

/home/shenweichen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
seed = 1024

path = './X_v2/'


def read_data(mode='train'):
    if mode=='train':
        df_basic = pd.read_pickle(path+'online_'+mode+'_x_26_29'+'.pkl.6.26')
    else:
        df_basic = pd.read_pickle(path + 'online_' + mode + '_x_31_31' + '.pkl.6.26')
    return df_basic

In [14]:
train_off = read_data('train')
test_off = read_data('test')
print(train_off.shape,test_off.shape)

(11947463, 134) (3321748, 133)


In [15]:
train_gl_trick = pd.read_pickle('./X_v2/train_on_trick_global.pkl')
test_gl_trick = pd.read_pickle('./X_v2/test_on_trick_global.pkl')

In [16]:
train_gl_trick.index = train_off.index
test_gl_trick.index = test_off.index
train_off = pd.concat([train_off,train_gl_trick],axis=1)
test_off = pd.concat([test_off,test_gl_trick],axis=1)

In [17]:
feature_names = [#'creativeID', 'userID',
                 'positionID', 'connectionType', 'telecomsOperator', 'age', 'gender',
                 'education', 'marriageStatus', 'haveBaby', 'ht_province',
                 'rd_province', 'sitesetID', 'positionType', 'adID',
                 'camgaignID', 'advertiserID', 'appID', 'appPlatform',
                 # 'user_start_install_cate_0',
                 'user_start_install_cate_1',
                 'user_start_install_cate_2', 'user_start_install_cate_3',
                 'user_start_install_cate_4', 'user_start_install_cate_5',

                 'appCategory', 'trick', 'first_diff', 'last_diff', 'user_hist_install', 'clickHour',
                 'user_adID_click_day', 'user_adID_click_hour',
                 'user_camgaignID_click_day', 'user_camgaignID_click_hour',
                 'user_appID_click_day', 'user_appID_click_hour',

                 'user_sitesetID_click_day',
                 'user_sitesetID_click_hour', 'user_click_day',

                 'positionID_cvr_smooth', 'creativeID_cvr', 'userID_cvr', 'adID_cvr', 'appID_cvr',
                 'positionID_sum_count', 'creativeID_sum_count', 'appID_sum_count',
                 'adID_sum_count', 'userID_sum_count',
                 'user_ad_click_day_mean', 'user_ad_click_day_min',
                 'user_camgaign_click_day_min', 'user_app_click_day_mean',
                 'user_app_click_day_max', 'user_app_click_day_min',
                 'user_site_click_day_mean', 'user_site_click_day_max',
                 'user_site_click_day_min', 'user_click_day_mean', 'user_click_day_max', 'user_click_day_min',
                 'advertiserID_user_clicks_age_cut_0',
                 'advertiserID_user_clicks_age_cut_1',
                 'advertiserID_user_clicks_age_cut_2',
                 'advertiserID_user_clicks_age_cut_3',
                 'advertiserID_user_clicks_age_cut_4',
                 'advertiserID_user_clicks_age_cut_5',
                 'advertiserID_user_clicks_age_cut_6',
                 'advertiserID_user_clicks_age_cut_7',
                 'advertiserID_user_clicks_gender_0',
                 'advertiserID_user_clicks_gender_1',
                 'advertiserID_user_clicks_gender_2',
                 'advertiserID_user_clicks_education_0',
                 'advertiserID_user_clicks_education_1',
                 'advertiserID_user_clicks_education_2',
                 'advertiserID_user_clicks_education_3',
                 'advertiserID_user_clicks_education_4',
                 'advertiserID_user_clicks_education_5',
                 'advertiserID_user_clicks_education_6',
                 'advertiserID_user_clicks_education_7',
                 'advertiserID_user_clicks_marriageStatus_0',
                 'advertiserID_user_clicks_marriageStatus_1',
                 'advertiserID_user_clicks_marriageStatus_2',
                 'advertiserID_user_clicks_marriageStatus_3',

                 'appID_user_clicks_age_cut_0', 'appID_user_clicks_age_cut_1',
                 'appID_user_clicks_age_cut_2', 'appID_user_clicks_age_cut_3',
                 'appID_user_clicks_age_cut_4', 'appID_user_clicks_age_cut_5',
                 'appID_user_clicks_age_cut_6', 'appID_user_clicks_age_cut_7',
                 'appID_user_clicks_gender_0', 'appID_user_clicks_gender_1',
                 'appID_user_clicks_gender_2', 'appID_user_clicks_education_0',
                 'appID_user_clicks_education_1', 'appID_user_clicks_education_2',
                 'appID_user_clicks_education_3', 'appID_user_clicks_education_4',
                 'appID_user_clicks_education_5', 'appID_user_clicks_education_6',
                 'appID_user_clicks_education_7', 'appID_user_clicks_marriageStatus_0',
                 'appID_user_clicks_marriageStatus_1',
                 'appID_user_clicks_marriageStatus_2',
                 'appID_user_clicks_marriageStatus_3',
                 'appID_user_clicks_haveBaby_0',
                 'appID_user_clicks_haveBaby_1', 'appID_user_clicks_haveBaby_2',
                 'appID_user_clicks_haveBaby_3', 'appID_user_clicks_haveBaby_4',
                 'appID_user_clicks_haveBaby_5', 'appID_user_clicks_haveBaby_6','install2click','global_uct_cnt','global_first',
'global_last']

In [18]:
cate_features = []

In [20]:
clf = LGBMClassifier(num_leaves=110, max_depth=12,
                     learning_rate=0.1, n_estimators=1200,
                     seed=0, nthread=20, subsample=0.8, colsample_bytree=0.9,
                     reg_lambda=0.005, )

In [24]:
clf.fit(train_off[feature_names],train_off['label'].values,
        eval_set=[(train_off[feature_names],train_off['label'])],
             feature_name=feature_names,categorical_feature=cate_features,
            verbose=10,
           )

[10]	valid_0's binary_logloss: 0.251229
[20]	valid_0's binary_logloss: 0.144581
[30]	valid_0's binary_logloss: 0.113862
[40]	valid_0's binary_logloss: 0.104689
[50]	valid_0's binary_logloss: 0.10171
[60]	valid_0's binary_logloss: 0.100512
[70]	valid_0's binary_logloss: 0.0999067
[80]	valid_0's binary_logloss: 0.0994477
[90]	valid_0's binary_logloss: 0.0990955
[100]	valid_0's binary_logloss: 0.098802
[110]	valid_0's binary_logloss: 0.0985379
[120]	valid_0's binary_logloss: 0.0983088
[130]	valid_0's binary_logloss: 0.0981337
[140]	valid_0's binary_logloss: 0.0979258
[150]	valid_0's binary_logloss: 0.0977876
[160]	valid_0's binary_logloss: 0.0976568
[170]	valid_0's binary_logloss: 0.0975255
[180]	valid_0's binary_logloss: 0.0973883
[190]	valid_0's binary_logloss: 0.0972647
[200]	valid_0's binary_logloss: 0.0971419
[210]	valid_0's binary_logloss: 0.0970419
[220]	valid_0's binary_logloss: 0.0969271
[230]	valid_0's binary_logloss: 0.0968247
[240]	valid_0's binary_logloss: 0.0967272
[250]	val

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.9, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=12,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=1200, nthread=20, num_leaves=110,
        objective='binary', reg_alpha=0, reg_lambda=0.005,
        scale_pos_weight=1, seed=0, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=0.8, subsample_for_bin=50000,
        subsample_freq=1, uniform_drop=False, xgboost_dart_mode=False)

In [26]:
pred = clf.predict_proba(test_off[feature_names],num_iteration=1200)[:,1]

In [28]:
np.mean(pred)

0.025097932123325013

In [29]:
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = pred
# result['prob'] = result['prob'].apply(adj)

result.to_csv('./res/lgb_final.csv')